In [1]:
import re
import numpy as np
import pandas as pd
import seaborn as sns

from pathlib import Path
from scipy import stats
from veliadb import base, settings
from veliadb import benchling_orm as bo
from veliadb.base import (Session, Orf, OrfXref, Transcript, Gene, 
                          TranscriptOrf, SequenceRegionXref, Protein, 
                          ProteinXref, Dataset, ProteinOrf)

from sqlalchemy.sql.expression import func, and_, or_

from dashboard import data_load
import pyarrow.parquet as pq
from dashboard.etl import CACHE_DIR, TPM_DESEQ2_FACTOR, DATA_DIR

pd.options.display.max_columns = 50
pd.options.display.max_colwidth = 100

2024-05-27 18:16:14.307 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-05-27 18:16:14.309 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-05-27 18:16:14.310 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-05-27 18:16:14.310 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-05-27 18:16:14.312 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-05-27 18:16:14.312 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-05-27 18:16:14.313 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-05-27 18:16:14.313 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-05-

In [3]:
session = base.Session()

In [4]:
session.query(Orf).count()

16282850

### Swissprot

In [35]:
swissprot_query = \
    session.query(Protein, Orf)\
           .join(ProteinXref, ProteinXref.protein_id == Protein.id)\
           .join(Dataset, Dataset.id == ProteinXref.xref_dataset_id)\
           .join(ProteinOrf, ProteinOrf.protein_id == Protein.id)\
           .join(Orf, Orf.id == ProteinOrf.orf_id)\
           .filter(Dataset.name == 'swissprot')\
           .filter(func.length(Protein.aa_seq) < 150)\
           .distinct(ProteinXref.protein_id)



In [32]:
alan_df = pd.read_csv('../../data/orfs_under_150codons_with_function-2.csv')

In [37]:
with open('../cache_updates/swissprot_sORF_150aa_vtx.txt', 'w') as outfile:
    for prot, orf in swissprot_query.all():
        outfile.write(f'VTX-{orf.id:07d}\n')
    

In [28]:
prot, orf = swissprot_query.first()

In [19]:
prot.

SyntaxError: invalid syntax (2621462385.py, line 1)

In [15]:
prot.ensembl_protein_id

'ENSP00000319240.2'

In [14]:
orf.ensembl_protein_id

'ENSP00000319240.2'

In [5]:
session.query(ProteinOrf).count()

0

In [19]:
df = pd.read_parquet(CACHE_DIR.joinpath('sorf_df.parq'))

In [20]:
signal_cols = ['SignalP 4.1_cut', 'SignalP 5b_cut', 'SignalP 6slow_cut', 'Deepsig_cut']
conservation_cols = ['tblastn_align_identity', 'blastp_align_identity', 'nonsig_blastp_align_identity', 'nonsig_tblastn_align_identity']
isoform_cols = ['swissprot_isoform', 'ensembl_isoform', 'refseq_isoform']
conservation_threshold = 50

measured_secreted_or_predicted_secreted =  (df[signal_cols] > -1).any(axis=1)

In [24]:
x = df[measured_secreted_or_predicted_secreted]

In [26]:
set(x['screening_phase'])

{'Not Screened',
 'Phase 1',
 'Phase 2',
 'Phase 3',
 'RefSeq;ENSEMBL;velia_phase5_uniprot-tremble',
 'RefSeq;velia_phase1_Chen;velia_phase1_Bona fide;velia_phase2_Bonafide_Bianca;velia_phase3_nan;velia_phase1_Prensner;ENSEMBL',
 'RefSeq;velia_phase5_uniprot-tremble',
 'velia_phase4_nan',
 'velia_phase4_nan;openprot',
 'velia_phase4_nan;velia_phase1_Chen',
 'velia_phase4_nan;velia_phase1_Chen;openprot',
 'velia_phase5_Blume_Mudge;velia_phase6_plasma_mass_spec',
 'velia_phase5_autoimmune lncRNA',
 'velia_phase5_autoimmune lncRNA;openprot',
 'velia_phase5_autoimmune lncRNA;velia_phase4_nan',
 'velia_phase5_uniprot-tremble',
 'velia_phase5_uniprot-tremble;openprot',
 'velia_phase6_autoimmune_gwas;openprot',
 'velia_phase6_public_mass_spec',
 'velia_phase6_public_mass_spec;openprot'}

In [30]:
x[x['screening_phase'] != 'Phase 3']

,show_details,vtx_id,aa_length,screening_phase_id,screening_phase,ucsc_track,source,orf_xrefs,protein_xrefs,gene_xrefs,transcript_xrefs,transcripts_exact,aa,nucl,index_copy,genscript_id,chr,strand,start,end,chrom_starts,block_sizes,phases,Ribo-Seq RPKM Support,blastp_score,...,Deepsig_cut,SignalP 6slow_cut,SignalP 5b_cut,SignalP 4.1_cut,Phobius,DeepTMHMM,nonsignal_seqs,trans1,trans2,trans3,sec1,sec2,sec3,translated_mean,secreted_mean,secreted,translated,swissprot_isoform,ensembl_isoform,refseq_isoform,phylocsf_58m_avg,phylocsf_58m_max,phylocsf_58m_min,phylocsf_vals,ESMFold plddt 90th percentile
vtx_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
VTX-0688940,False,VTX-0688940,126,-1,Not Screened,chr4:54658015-54695825,[],[],[],ENSG00000157404.17;HGNC:6342;KIT,ENST00000514582.1;OTTHUMT00000361418.1,[ENST00000514582],MRGARGAWDFLCVLLLLLRVQTGSSQPSVSPGEPSPPSIHPGKSDLIVRVGDEIRLLCTDPGFVKWTFEILDETNENKQNEWITEKAEATNTGKYT...,ATGAGAGGCGCTCGCGGCGCCTGGGATTTTCTCTGCGTTCTGCTCCTACTGCTTCGCGTCCAGACAGGCTCTTCTCAACCATCTGTGAGTCCAGGG...,VTX-0688940,,chr4,+,54658015,54695825,54658015;54695512,67;314,0;2,True,389.0,...,25,25,25,25,-1,25,-1.0,NaN,NaN,NaN,NaN,None,None,NaN,NaN,True,True,None,None,None,1.045192,10.397,-8.702,"[-0.17299999296665192, -0.17299999296665192, -0.17299999296665192, -0.1679999977350235, -0.16799...",69.523293
VTX-0826553,False,VTX-0826553,96,-1,Not Screened,chr9:128149526-128150527,[],[],[],ENSG00000148346.12;HGNC:6526;LCN2,ENST00000470902.5;OTTHUMT00000054376.1;ENST00000487719.1;OTTHUMT00000054379.1,"[ENST00000470902, ENST00000487719]",MPLGLLWLGLALLGALHAQAQDSTSDLIPAPPLSKVPLQQNFQDNQVRADISWGCESQTDVTGKGWPKLRDPVVHLAVLPGIHLCSSFLCSLEQRL,ATGCCCCTAGGTCTCCTGTGGCTGGGCCTAGCCCTGTTGGGGGCTCTGCATGCCCAGGCCCAGGACTCCACCTCAGACCTGATCCCAGCCCCACCT...,VTX-0826553,,chr9,+,128149526,128150527,128149526;128150375,138;153,0;0,True,64.0,...,20,20,20,20,-1,18,-1.0,47270.0,53510.0,30520.0,21700.0,22370,99480.0,43766.666667,21700.0,True,True,None,None,None,-4.595113,6.380,-14.386,"[-3.259000062942505, -3.259000062942505, -3.259000062942505, -6.340000152587891, -6.340000152587...",43.645977
VTX-0846585,False,VTX-0846585,50,-1,Not Screened,chrX:153794567-153795715,[],[],[],ENSG00000180879.14;HGNC:11326;SSR4,ENST00000482902.5;OTTHUMT00000061031.1;11326;CHS.59049.11;NR_037927.1;RefSeq:NR_037927.1;rna-NR_...,[ENST00000482902],MPAQPFGREEAMAAMASLGALALLLLSSLSRCSGICLLWSRPRAFSDVLC,ATGCCGGCCCAGCCGTTCGGCAGAGAAGAGGCGATGGCGGCGATGGCATCTCTCGGCGCCCTGGCGCTGCTCCTGCTGTCCAGCCTCTCCCGCTGC...,VTX-0846585,,chrX,+,153794567,153795715,153794567;153794674;153795663,19;81;53,0;2;2,False,NaN,...,-1,34,-1,-1,-1,30,-1.0,NaN,NaN,NaN,NaN,None,None,NaN,NaN,False,None,None,None,None,NaN,NaN,NaN,None,67.050358
VTX-0738397,False,VTX-0738397,34,-1,Not Screened,chr6:32166166-32166270,[],[],[],ENSG00000258388.7;HGNC:48343;PPT2-EGFL8;13944;80864;EGFL8;EGFL8;ENSG00000241404.7;gene-EGFL8;HGN...,ENST00000421600.2;OTTHUMT00000412522.2;ENST00000428388.6;OTTHUMT00000412520.1;ENST00000453656.6;...,"[ENST00000421600, ENST00000428388, ENST00000453656, ENST00000466239, ENST00000583227]",MGSRAELCTLLGGFSFLLLLIPGEGAKGGSLRER,ATGGGGTCCAGGGCTGAGCTGTGCACTCTCTTAGGCGGATTCTCCTTCCTCCTGCTACTGATACCAGGCGAGGGGGCCAAGGGTGGATCCCTCAGA...,VTX-0738397,,chr6,+,32166166,32166270,32166166,105,0,False,86.0,...,25,-1,25,-1,-1,25,-1.0,710.0,670.0,580.0,10.0,10,20.0,653.333333,10.0,False,False,None,None,None,-4.481629,-2.070,-8.226,"[-3.309999942779541, -3.309999942779541, -3.309999942779541, -4.703999996185303, -4.703999996185...",65.348766
VTX-0849971,False,VTX-0849971,43,-1,Phase 2,chr3:107526338-107645906,[],[],"[BBX_107245184_43aa, c3riboseqorf111_GENE_ID=ENSG00000114439_TYPE=uORF_LEN=132, Phase 2_465, Pha...",BBX;ENSG00000114439.19;HGNC:14422,14422;BBX;CCDS46881.1;CHS.37927.9;ENST00000325805.13;NM_001142568.3;OTTHUMT00000317820.2;RefSeq:...,"[ENST00000325805, ENST00000402163, ENST00000402543, ENST00000415149, ENST00000416476, ENST000004...",MTKALPQFIFLPVYFPFAFLESCCITEAGSSDVPLKSQWKVLT,ATGACAAAGGCTTTGCCGCAGTTCATCTTCCTCCCTGTGTACTTT

In [28]:
x

,show_details,vtx_id,aa_length,screening_phase_id,screening_phase,ucsc_track,source,orf_xrefs,protein_xrefs,gene_xrefs,transcript_xrefs,transcripts_exact,aa,nucl,index_copy,genscript_id,chr,strand,start,end,chrom_starts,block_sizes,phases,Ribo-Seq RPKM Support,blastp_score,...,Deepsig_cut,SignalP 6slow_cut,SignalP 5b_cut,SignalP 4.1_cut,Phobius,DeepTMHMM,nonsignal_seqs,trans1,trans2,trans3,sec1,sec2,sec3,translated_mean,secreted_mean,secreted,translated,swissprot_isoform,ensembl_isoform,refseq_isoform,phylocsf_58m_avg,phylocsf_58m_max,phylocsf_58m_min,phylocsf_vals,ESMFold plddt 90th percentile
vtx_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
VTX-0688940,False,VTX-0688940,126,-1,Not Screened,chr4:54658015-54695825,[],[],[],ENSG00000157404.17;HGNC:6342;KIT,ENST00000514582.1;OTTHUMT00000361418.1,[ENST00000514582],MRGARGAWDFLCVLLLLLRVQTGSSQPSVSPGEPSPPSIHPGKSDLIVRVGDEIRLLCTDPGFVKWTFEILDETNENKQNEWITEKAEATNTGKYT...,ATGAGAGGCGCTCGCGGCGCCTGGGATTTTCTCTGCGTTCTGCTCCTACTGCTTCGCGTCCAGACAGGCTCTTCTCAACCATCTGTGAGTCCAGGG...,VTX-0688940,,chr4,+,54658015,54695825,54658015;54695512,67;314,0;2,True,389.0,...,25,25,25,25,-1,25,-1.0,NaN,NaN,NaN,NaN,None,None,NaN,NaN,True,True,None,None,None,1.045192,10.397,-8.702,"[-0.17299999296665192, -0.17299999296665192, -0.17299999296665192, -0.1679999977350235, -0.16799...",69.523293
VTX-0826553,False,VTX-0826553,96,-1,Not Screened,chr9:128149526-128150527,[],[],[],ENSG00000148346.12;HGNC:6526;LCN2,ENST00000470902.5;OTTHUMT00000054376.1;ENST00000487719.1;OTTHUMT00000054379.1,"[ENST00000470902, ENST00000487719]",MPLGLLWLGLALLGALHAQAQDSTSDLIPAPPLSKVPLQQNFQDNQVRADISWGCESQTDVTGKGWPKLRDPVVHLAVLPGIHLCSSFLCSLEQRL,ATGCCCCTAGGTCTCCTGTGGCTGGGCCTAGCCCTGTTGGGGGCTCTGCATGCCCAGGCCCAGGACTCCACCTCAGACCTGATCCCAGCCCCACCT...,VTX-0826553,,chr9,+,128149526,128150527,128149526;128150375,138;153,0;0,True,64.0,...,20,20,20,20,-1,18,-1.0,47270.0,53510.0,30520.0,21700.0,22370,99480.0,43766.666667,21700.0,True,True,None,None,None,-4.595113,6.380,-14.386,"[-3.259000062942505, -3.259000062942505, -3.259000062942505, -6.340000152587891, -6.340000152587...",43.645977
VTX-0846585,False,VTX-0846585,50,-1,Not Screened,chrX:153794567-153795715,[],[],[],ENSG00000180879.14;HGNC:11326;SSR4,ENST00000482902.5;OTTHUMT00000061031.1;11326;CHS.59049.11;NR_037927.1;RefSeq:NR_037927.1;rna-NR_...,[ENST00000482902],MPAQPFGREEAMAAMASLGALALLLLSSLSRCSGICLLWSRPRAFSDVLC,ATGCCGGCCCAGCCGTTCGGCAGAGAAGAGGCGATGGCGGCGATGGCATCTCTCGGCGCCCTGGCGCTGCTCCTGCTGTCCAGCCTCTCCCGCTGC...,VTX-0846585,,chrX,+,153794567,153795715,153794567;153794674;153795663,19;81;53,0;2;2,False,NaN,...,-1,34,-1,-1,-1,30,-1.0,NaN,NaN,NaN,NaN,None,None,NaN,NaN,False,None,None,None,None,NaN,NaN,NaN,None,67.050358
VTX-0738397,False,VTX-0738397,34,-1,Not Screened,chr6:32166166-32166270,[],[],[],ENSG00000258388.7;HGNC:48343;PPT2-EGFL8;13944;80864;EGFL8;EGFL8;ENSG00000241404.7;gene-EGFL8;HGN...,ENST00000421600.2;OTTHUMT00000412522.2;ENST00000428388.6;OTTHUMT00000412520.1;ENST00000453656.6;...,"[ENST00000421600, ENST00000428388, ENST00000453656, ENST00000466239, ENST00000583227]",MGSRAELCTLLGGFSFLLLLIPGEGAKGGSLRER,ATGGGGTCCAGGGCTGAGCTGTGCACTCTCTTAGGCGGATTCTCCTTCCTCCTGCTACTGATACCAGGCGAGGGGGCCAAGGGTGGATCCCTCAGA...,VTX-0738397,,chr6,+,32166166,32166270,32166166,105,0,False,86.0,...,25,-1,25,-1,-1,25,-1.0,710.0,670.0,580.0,10.0,10,20.0,653.333333,10.0,False,False,None,None,None,-4.481629,-2.070,-8.226,"[-3.309999942779541, -3.309999942779541, -3.309999942779541, -4.703999996185303, -4.703999996185...",65.348766
VTX-0849971,False,VTX-0849971,43,-1,Phase 2,chr3:107526338-107645906,[],[],"[BBX_107245184_43aa, c3riboseqorf111_GENE_ID=ENSG00000114439_TYPE=uORF_LEN=132, Phase 2_465, Pha...",BBX;ENSG00000114439.19;HGNC:14422,14422;BBX;CCDS46881.1;CHS.37927.9;ENST00000325805.13;NM_001142568.3;OTTHUMT00000317820.2;RefSeq:...,"[ENST00000325805, ENST00000402163, ENST00000402543, ENST00000415149, ENST00000416476, ENST000004...",MTKALPQFIFLPVYFPFAFLESCCITEAGSSDVPLKSQWKVLT,ATGACAAAGGCTTTGCCGCAGTTCATCTTCCTCCCTGTGTACTTT